In [3]:
import configparser
import logging
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark import SparkContext

In [6]:
# setting up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


In [7]:
# AWS configuration
base = os.path.abspath("__file__" + "/../")  # get base path
config = configparser.ConfigParser()
config.read(os.path.join(base +
                         '/variables.cfg'))


['c:\\Users\\ofili\\Documents\\Projects\\Data Engineering\\nyc-taxi\\scripts/variables.cfg']

In [8]:
key = config.get('AWS', 'AWS_ACCESS_KEY_ID')
secret = config.get('AWS', 'AWS_SECRET_ACCESS_KEY')


In [ ]:
# read data from s3
df_green = spark.read.format("parquet").load(
    "s3a://nyc-taxi-data-1/raw/green/green_tripdata_2015-01.parquet")

In [29]:
# spark session
spark = SparkSession.builder.appName(
    'Check Pyspark').master("local").getOrCreate()


RuntimeError: Java gateway process exited before sending its port number

In [ ]:
try:
    aws_access_key_id = key
    aws_secret_access_key = secret

    spark = SparkSession.builder \
        .config("spark.jars.packages",
                "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .enableHiveSupport().getOrCreate()
    spark.sparkContext._jsc.hadoopConfiguration().set(
        "fs.s3a.access.key", aws_access_key_id)
    spark.sparkContext._jsc.hadoopConfiguration().set(
        "fs.s3a.secret.key", aws_secret_access_key)
    spark.sparkContext._jsc.hadoopConfiguration().set(
        "fs.s3a.endpoint", "s3.us-west-2.amazonaws.com")
    logger.info("Spark session created")
except Exception as spark_err:
    logger.error("Error creating spark session: %s", str(spark_err))
    raise spark_err

In [23]:
from analysis import create_spark_session

In [ ]:
s

In [26]:
df_yellow = spark.read.format("parquet").load(
    "data/yellow_tripdata_2019-01.parquet")


AttributeError: 'Builder' object has no attribute 'read'

In [ ]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-01-01 01:46:40|  2019-01-01 01:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.6

In [ ]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [ ]:
df.count()

NameError: name 'df' is not defined

In [ ]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))


In [ ]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))


In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)


In [ ]:
df_trips_data.groupBy('service_type').count().show()


In [3]:
df_yellow.registerTempTable('trips_data')


NameError: name 'df_yellow' is not defined

In [ ]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()


In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")


In [ ]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')


In [33]:
key = 'raw/green/*/*.parquet'

m = key.split('/')[1]
print(m)

green
